In [ ]:
import sagas.crmsfa.odoo_facade
import imp
imp.reload(sagas.crmsfa.odoo_facade)

In [1]:
from sagas.tool.app_settings import setup_logger, setup_jupyter
setup_logger('crmsfa.log')
setup_jupyter()

In [16]:
from sagas.crmsfa.odoo_facade import odoo, login
# login()
login(db="demo", username="admin", password="admin")

if 'sale.order' in odoo.env:
    Order = odoo.env['sale.order']
    order_ids = Order.search([])
    print(order_ids)
else:
    print("no sale.order model")

[8, 7, 6, 4, 3, 14, 9, 15, 10, 16, 11, 17, 12, 20, 19, 5, 2, 1, 18, 13]


* 显示当前登录的用户的信息

In [3]:
user = odoo.env.user
print(user.name)            # name of the user connected
print(user.company_id.name) # the name of its company

samlet@sagas.ai
YourCompany


In [4]:
Country = odoo.env['res.country']
de_id = Country.search([('code', '=', 'DE')])[0]
de = Country.browse(de_id)
print(de)
print(de.name)

Recordset('res.country', [57])
Germany


* 显示所有国家(或地区)对应的编号

In [5]:
rs_ids=Country.search([])
rs=Country.browse(rs_ids)
for el in rs:
    print(el.code, el.name)

AF Afghanistan
AL Albania
DZ Algeria
AS American Samoa
AD Andorra
AO Angola
AI Anguilla
AQ Antarctica
AG Antigua and Barbuda
AR Argentina
AM Armenia
AW Aruba
AU Australia
AT Austria
AZ Azerbaijan
BS Bahamas
BH Bahrain
BD Bangladesh
BB Barbados
BY Belarus
BE Belgium
BZ Belize
BJ Benin
BM Bermuda
BT Bhutan
BO Bolivia
BQ Bonaire, Sint Eustatius and Saba
BA Bosnia and Herzegovina
BW Botswana
BV Bouvet Island
BR Brazil
IO British Indian Ocean Territory
BN Brunei Darussalam
BG Bulgaria
BF Burkina Faso
BI Burundi
KH Cambodia
CM Cameroon
CA Canada
CV Cape Verde
KY Cayman Islands
CF Central African Republic
TD Chad
CL Chile
CN China
CX Christmas Island
CC Cocos (Keeling) Islands
CO Colombia
KM Comoros
CG Congo
CK Cook Islands
CR Costa Rica
HR Croatia
CU Cuba
CW Curaçao
CY Cyprus
CZ Czech Republic
CI Côte d'Ivoire
CD Democratic Republic of the Congo
DK Denmark
DJ Djibouti
DM Dominica
DO Dominican Republic
EC Ecuador
EG Egypt
SV El Salvador
GQ Equatorial Guinea
ER Eritrea
EE Estonia
ET Ethiopia
F

* 显示odoo中指定模型的信息

In [6]:
import odoorpc.fields as fields
from odoorpc.env import FIELDS_RESERVED
from tabulate import tabulate
import wcwidth

def desc_model(model):
    table_header = ['name','type', 'string']
    table_data = []

    attrs = {
                '_name': model,
                '_columns': {},
            }
    fields_get = odoo.execute(model, 'fields_get')
    for field_name, field_data in fields_get.items():
        if field_name not in FIELDS_RESERVED:
            Field = fields.generate_field(field_name, field_data)
            attrs['_columns'][field_name] = Field
            attrs[field_name] = Field
            # print(field_name, "%", field_data['type'])
            # print(field_data.keys())
            # ['searchable', 'sortable', 'depends', 'store', 
            # 'manual', 'type', 'company_dependent', 
            # 'change_default', 'readonly', 'string', 'required']
            table_data.append((field_name, field_data['type'], 
                               field_data['string']))
            # addi: type(Field)

    # print(tabulate(table_data, headers=table_header, tablefmt='grid'))
    # print(tabulate(table_data, headers=table_header, tablefmt='fancy_grid'))
    print(tabulate(table_data, headers=table_header, tablefmt='psql'))

model="res.country"
desc_model(model)

+-------------------+-----------+------------------------+
| name              | type      | string                 |
|-------------------+-----------+------------------------|
| name              | char      | Country Name           |
| code              | char      | Country Code           |
| address_format    | text      | Layout in Reports      |
| address_view_id   | many2one  | Input View             |
| currency_id       | many2one  | Currency               |
| image             | binary    | Image                  |
| phone_code        | integer   | Country Calling Code   |
| country_group_ids | many2many | Country Groups         |
| state_ids         | one2many  | States                 |
| name_position     | selection | Customer Name Position |
| vat_label         | char      | Vat Label              |
| display_name      | char      | Display Name           |
| create_uid        | many2one  | Created by             |
| create_date       | datetime  | Created on            

In [8]:
desc_model('mail.thread')

+----------------------------+-----------+-------------------------+
| name                       | type      | string                  |
|----------------------------+-----------+-------------------------|
| message_is_follower        | boolean   | Is Follower             |
| message_follower_ids       | one2many  | Followers               |
| message_partner_ids        | many2many | Followers (Partners)    |
| message_channel_ids        | many2many | Followers (Channels)    |
| message_ids                | one2many  | Messages                |
| message_unread             | boolean   | Unread Messages         |
| message_unread_counter     | integer   | Unread Messages Counter |
| message_needaction         | boolean   | Action Needed           |
| message_needaction_counter | integer   | Number of Actions       |
| message_has_error          | boolean   | Message Delivery error  |
| message_has_error_counter  | integer   | Number of error         |
| message_attachment_count   | int

In [9]:
# 显示Crm模块中的潜在用户数据表的字段信息
desc_model('crm.lead')

+----------------------------+-----------+-------------------------+
| name                       | type      | string                  |
|----------------------------+-----------+-------------------------|
| name                       | char      | Opportunity             |
| partner_id                 | many2one  | Customer                |
| active                     | boolean   | Active                  |
| date_action_last           | datetime  | Last Action             |
| email_from                 | char      | Email                   |
| website                    | char      | Website                 |
| team_id                    | many2one  | Sales Team              |
| kanban_state               | selection | Kanban State            |
| email_cc                   | text      | Global CC               |
| description                | text      | Notes                   |
| tag_ids                    | many2many | Tags                    |
| contact_name               | cha

In [11]:
model="res.partner"
model_exists = odoo.execute('ir.model', 'search',
                                          [('model', '=', model)])
print(bool(model_exists))

True


In [15]:
from tabulate import tabulate
import wcwidth
from sagas.crmsfa.odoo_facade import odoo, login

def model_info():
    # login()
    login(db="demo", username="admin", password="admin")

    # models = odoo.execute('ir.model', 'browse', [])
    # print(models)

    # https://stackoverflow.com/questions/2872512/python-truncate-a-long-string
    def abbrev(data, l=15):
        info = (data[:l] + '..') if len(data) > l else data
        return info

    table_header = ['model','name', 'count']
    table_data = []

    Order = odoo.env['ir.model']
    order_ids = Order.search([])
    print(len(order_ids))
    models=Order.browse(order_ids)
    # print(models[0])
    # odoo/addons/base/ir/ir_model.py
    for order in models:
        # print(order.model, order.name, order.count)    
        table_data.append((abbrev(order.model), 
                           abbrev(order.name), order.count))

    print(tabulate(table_data, headers=table_header, tablefmt='psql'))    

model_info()

384
+-------------------+----------------------------------+---------+
| model             | name                             |   count |
|-------------------+----------------------------------+---------|
| mail.thread       | EMail线程                        |       0 |
| hr.department     | HR 部门                          |       5 |
| product.product   | 产品                             |      49 |
| product.templat.. | 产品模板                         |      43 |
| account.payment   | 付款                             |       1 |
| project.task      | 任务                             |      31 |
| hr.leave          | 休假                             |       2 |
| hr.leave.alloca.. | 休假分配                         |       4 |
| account.analyti.. | 分析账户                         |      18 |
| account.invoice   | 发票                             |       8 |
| hr.employee       | 员工                             |      20 |
| hr.job            | 工作岗位                         |       7 |
| mrp.workorder     | 工

In [4]:
tabulate.WIDE_CHARS_MODE=True